In [ ]:
from google.colab import files
import pandas as pd
import unicodedata
import re
from collections import OrderedDict

In [ ]:
print("Upload FILE 1 (combos Excel) — contains sheet: data brutes (or Data brutes)")
up1 = files.upload()
file_combos = next(iter(up1.keys()))
print("Combos file:", file_combos)

print("\nUpload FILE 2 (mapping Excel) — contains sheet: ARV_TO_DCI")
up2 = files.upload()
file_map = next(iter(up2.keys()))
print("Mapping file:", file_map)


Upload FILE 1 (combos Excel) — contains sheet: data brutes (or Data brutes)


Saving ARV_JL_MBVR (1).xlsx to ARV_JL_MBVR (1) (4).xlsx
Combos file: ARV_JL_MBVR (1) (4).xlsx

Upload FILE 2 (mapping Excel) — contains sheet: ARV_TO_DCI


Saving data.xlsx to data (4).xlsx
Mapping file: data (4).xlsx


In [ ]:
def remove_accents(text: str) -> str:
    if pd.isna(text):
        return ""
    text = str(text)
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def normalize_token(s: str) -> str:
    s = remove_accents(s)
    s = str(s).upper().strip()
    # remove content in parentheses if present
    s = re.sub(r"\([^)]*\)", "", s)
    # remove all spaces
    s = re.sub(r"\s+", "", s)
    # remove any non alphanumeric characters (®, -, etc.)
    s = re.sub(r"[^A-Z0-9]", "", s)
    return s


def normalize_dci_cell(s: str) -> str:
    s = remove_accents(s)
    s = s.upper().strip()
    # normalize separators to comma
    s = s.replace(";", ",").replace("/", ",").replace("|", ",").replace("+", ",")
    s = re.sub(r"\s*,\s*", ",", s)
    return s

def split_dci_list(dci_cell: str) -> list:
    if pd.isna(dci_cell) or str(dci_cell).strip() == "":
        return []
    s = normalize_dci_cell(dci_cell)
    return [p.strip() for p in s.split(",") if p.strip()]

def unique_preserve_order(seq):
    seen = set()
    out = []
    for x in seq:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

def extract_inside_parentheses(token: str) -> str:
    """
    If token has '(...)', keep what's inside.
    Example: 'Y(W)' -> 'W'
    """
    m = re.search(r"\(([^)]*)\)", token)
    if m:
        return m.group(1)
    return token

def split_combo_specialites(raw_combo: str) -> list:
    """
    Rules:
    1) General case: split combo into parts using + (also supports / ; |)
       - if a part contains parentheses -> take inside parentheses
       - else take the part itself

    2) Special case (your new request):
       If the raw combo contains '/' AND contains parentheses '(...)',
       example: X/Y(W)/Z
       -> ONLY keep the inside-parentheses value: W
    """
    if pd.isna(raw_combo) or str(raw_combo).strip() == "":
        return []

    raw = str(raw_combo)
    raw_no_acc = remove_accents(raw).upper()

    # ✅ SPECIAL CASE: X/Y(W)/Z  -> only W
    # (contains '/' and at least one '(...)')
    if "/" in raw_no_acc and re.search(r"\(([^)]*)\)", raw_no_acc):
        inside = re.findall(r"\(([^)]*)\)", raw_no_acc)  # could be multiple
        # keep all inside values (usually 1), normalized
        meds = [normalize_token(x) for x in inside if normalize_token(x)]
        # your requirement says "the only specialité is W"
        # if multiple parentheses exist, we keep all inside parentheses (safe)
        return meds

    # Otherwise: normal behavior
    s = raw_no_acc
    # normalize separators to '+'
    s = re.sub(r"[\/|;]", "+", s)

    parts = [p.strip() for p in s.split("+") if p.strip()]

    meds = []
    for part in parts:
        chosen = extract_inside_parentheses(part)   # keeps W for Y(W)
        chosen = normalize_token(chosen)
        if chosen:
            meds.append(chosen)

    return meds


In [ ]:
# Load all sheet names and pick the correct one
sheets_combos = pd.read_excel(file_combos, sheet_name=None)
print("Sheets in combos file:", list(sheets_combos.keys()))

sheet_data_brutes = None
for s in sheets_combos.keys():
    if s.strip().lower() == "data brutes":
        sheet_data_brutes = s
        break

if sheet_data_brutes is None:
    raise ValueError("Sheet 'data brutes' not found (case/spacing). Check your combos file sheets.")

df_combos_raw = sheets_combos[sheet_data_brutes]
print("Combos sheet columns:", list(df_combos_raw.columns))
df_combos_raw.head()


Sheets in combos file: ['Data brutes', 'Analyse']
Combos sheet columns: ['ARV présents dans NADIS']


,ARV présents dans NADIS
0,Combivir + Kalétra
1,Isentress + Truvada
2,Isentress + Kivexa
3,Triumeq
4,Genvoya


In [ ]:
COMBO_COL = df_combos_raw.columns[0]   # "ARV présents dans NADIS" in your case
print("Using combo column:", COMBO_COL)

df_combos = df_combos_raw.copy()
df_combos["combo_specialites_parsed"] = df_combos[COMBO_COL].apply(
    lambda x: "+".join(split_combo_specialites(x))
)

df_combos = df_combos[df_combos["combo_specialites_parsed"] != ""].drop_duplicates(subset=["combo_specialites_parsed"])
df_combos.head(20)


Using combo column: ARV présents dans NADIS


,ARV présents dans NADIS,combo_specialites_parsed
0,Combivir + Kalétra,COMBIVIR+KALETRA
1,Isentress + Truvada,ISENTRESS+TRUVADA
2,Isentress + Kivexa,ISENTRESS+KIVEXA
3,Triumeq,TRIUMEQ
4,Genvoya,GENVOYA
5,Biktarvy,BIKTARVY
6,Lamivudine / Dolutégravir (Dovato),DOVATO
7,Rilpivirine / Dolutégravir (Juluca),JULUCA
8,Retrovir + Videx,RETROVIR+VIDEX
9,Retrovir + Videx + Invirase,RETROVIR+VIDEX+INVIRASE


In [ ]:
# --- Read all sheets from mapping file (file 2) ---
sheets_map = pd.read_excel(file_map, sheet_name=None)
print("Sheets in mapping file:", list(sheets_map.keys()))

# --- Find ARV_TO_DCI sheet robustly (case/spacing safe) ---
sheet_arv_to_dci = None
for s in sheets_map.keys():
    if s.strip().lower() == "arv_to_dci":
        sheet_arv_to_dci = s
        break

if sheet_arv_to_dci is None:
    raise ValueError("Sheet 'ARV_TO_DCI' not found. Check your mapping file sheets.")

df_map_raw = sheets_map[sheet_arv_to_dci]
print("Mapping sheet columns:", list(df_map_raw.columns))
display(df_map_raw.head())

# ✅ Use the columns you requested
SPECIALITE_COL = "EXTRACTED_VALUE"
DCI_COL = "DCI_ASSOCIES"

# Validate columns exist
missing_cols = [c for c in [SPECIALITE_COL, DCI_COL] if c not in df_map_raw.columns]
if missing_cols:
    raise ValueError(f"Missing required columns in ARV_TO_DCI: {missing_cols}")

# --- Build mapping dict: specialité -> list of DCIs ---
df_map = df_map_raw.copy()
df_map["specialite_clean"] = df_map[SPECIALITE_COL].apply(normalize_token)
df_map["dcis_list"] = df_map[DCI_COL].apply(split_dci_list)

spec_to_dcis = {}
for _, row in df_map.iterrows():
    sp = row["specialite_clean"]
    dcis = row["dcis_list"]

    if not sp:
        continue

    spec_to_dcis.setdefault(sp, []).extend(dcis)

# Deduplicate DCIs per specialité while preserving order
for sp in spec_to_dcis:
    spec_to_dcis[sp] = unique_preserve_order(spec_to_dcis[sp])

print("Sample mapping (specialité -> DCIs):")
print(list(spec_to_dcis.items())[:10])


Sheets in mapping file: ['ARV_BRUTES', 'DCI_SPECIALITE', 'ARV_TO_DCI', 'LISTE_DCI']
Mapping sheet columns: ['ARV_BRUTES', 'EXTRACTED_VALUE', 'DCI_ASSOCIES', 'Unnamed: 3']


,ARV_BRUTES,EXTRACTED_VALUE,DCI_ASSOCIES,Unnamed: 3
0,COMBIVIR,COMBIVIR,"LAMIVUDINE, ZIDOVUDINE",NaN
1,KALETRA,KALETRA,"LOPINAVIR, RITONAVIR",NaN
2,ISENTRESS,ISENTRESS,RALTEGRAVIR,NaN
3,TRUVADA,TRUVADA,"EMTRICITABINE, TENOFOVIR DISOPROXIL FUMARATE",NaN
4,KIVEXA,KIVEXA,"ABACAVIR, LAMIVUDINE",NaN


Sample mapping (specialité -> DCIs):
[('COMBIVIR', ['LAMIVUDINE', 'ZIDOVUDINE']), ('KALETRA', ['LOPINAVIR', 'RITONAVIR']), ('ISENTRESS', ['RALTEGRAVIR']), ('TRUVADA', ['EMTRICITABINE', 'TENOFOVIR DISOPROXIL FUMARATE']), ('KIVEXA', ['ABACAVIR', 'LAMIVUDINE']), ('TRIUMEQ', ['ABACAVIR', 'DOLUTEGRAVIR', 'LAMIVUDINE']), ('GENVOYA', ['COBICISTAT', 'ELVITEGRAVIR', 'EMTRICITABINE', 'TENOFOVIR ALAFENAMIDE']), ('BIKTARVY', ['BICTEGRAVIR', 'EMTRICITABINE', 'TENOFOVIR ALAFENAMIDE']), ('DOVATO', ['DOLUTEGRAVIR', 'LAMIVUDINE']), ('JULUCA', ['DOLUTEGRAVIR', 'RILPIVIRINE'])]


In [ ]:
SPECIALITE_COL = "ARV_BRUTES"
DCI_COL = "DCI_ASSOCIES"   # ✅ FIXED

if SPECIALITE_COL not in df_map_raw.columns or DCI_COL not in df_map_raw.columns:
    raise ValueError(f"Expected columns not found. Need '{SPECIALITE_COL}' and '{DCI_COL}'.")

df_map = df_map_raw.copy()
df_map["specialite_clean"] = df_map[SPECIALITE_COL].apply(normalize_token)
df_map["dcis_list"] = df_map[DCI_COL].apply(split_dci_list)

spec_to_dcis = {}
for _, row in df_map.iterrows():
    sp = row["specialite_clean"]
    dcis = row["dcis_list"]

    if not sp:
        continue

    spec_to_dcis.setdefault(sp, []).extend(dcis)

# Deduplicate per specialité
for sp in spec_to_dcis:
    spec_to_dcis[sp] = unique_preserve_order(spec_to_dcis[sp])

# sanity check: should show real DCIs now
list(spec_to_dcis.items())[:5]


[('COMBIVIR', ['LAMIVUDINE', 'ZIDOVUDINE']),
 ('KALETRA', ['LOPINAVIR', 'RITONAVIR']),
 ('ISENTRESS', ['RALTEGRAVIR']),
 ('TRUVADA', ['EMTRICITABINE', 'TENOFOVIR DISOPROXIL FUMARATE']),
 ('KIVEXA', ['ABACAVIR', 'LAMIVUDINE'])]

In [ ]:
rows = []

for _, r in df_combos.iterrows():
    combo_original = r[COMBO_COL]
    combo_parsed = r["combo_specialites_parsed"]
    meds = combo_parsed.split("+") if combo_parsed else []

    all_dcis = []
    missing = []

    for med in meds:
        if med in spec_to_dcis:
            all_dcis.extend(spec_to_dcis[med])
        else:
            missing.append(med)

    all_dcis = unique_preserve_order(all_dcis)

    rows.append({
        "combo_original": combo_original,
        "combo_specialites": combo_parsed,
        "combo_dcis": "+".join(all_dcis),
        "missing_specialites": "+".join(missing)
    })

df_combo_result = pd.DataFrame(rows).drop_duplicates(subset=["combo_specialites"])
df_combo_result.head(30)


,combo_original,combo_specialites,combo_dcis,missing_specialites
0,Combivir + Kalétra,COMBIVIR+KALETRA,LAMIVUDINE+ZIDOVUDINE+LOPINAVIR+RITONAVIR,
1,Isentress + Truvada,ISENTRESS+TRUVADA,RALTEGRAVIR+EMTRICITABINE+TENOFOVIR DISOPROXIL...,
2,Isentress + Kivexa,ISENTRESS+KIVEXA,RALTEGRAVIR+ABACAVIR+LAMIVUDINE,
3,Triumeq,TRIUMEQ,ABACAVIR+DOLUTEGRAVIR+LAMIVUDINE,
4,Genvoya,GENVOYA,COBICISTAT+ELVITEGRAVIR+EMTRICITABINE+TENOFOVI...,
5,Biktarvy,BIKTARVY,BICTEGRAVIR+EMTRICITABINE+TENOFOVIR ALAFENAMIDE,
6,Lamivudine / Dolutégravir (Dovato),DOVATO,DOLUTEGRAVIR+LAMIVUDINE,
7,Rilpivirine / Dolutégravir (Juluca),JULUCA,DOLUTEGRAVIR+RILPIVIRINE,
8,Retrovir + Videx,RETROVIR+VIDEX,ZIDOVUDINE+DIDANOSINE,
9,Retrovir + Videx + Invirase,RETROVIR+VIDEX+INVIRASE,ZIDOVUDINE+DIDANOSINE+SAQUINAVIR,


In [ ]:
OUTPUT_FILE = "mapping_with_combo_dcis.xlsx"
NEW_SHEET_NAME = "combo_specialite_to_dci"

# We already loaded sheets_map (all sheets from file 2)
# Just add/replace the new sheet:
sheets_map[NEW_SHEET_NAME] = df_combo_result

with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:
    for sheet_name, df in sheets_map.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Saved:", OUTPUT_FILE)
files.download(OUTPUT_FILE)


Saved: mapping_with_combo_dcis.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>